In [ ]:
%load_ext autoreload
%autoreload 2

In [2]:
!nvidia-smi

Sat Apr 26 16:42:17 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-80GB          On  |   00000000:C1:00.0 Off |                    0 |
| N/A   25C    P0             61W /  500W |       1MiB /  81920MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [3]:
!pip install torch

In [4]:
!pip install datasets

  Using cached datasets-3.5.0-py3-none-any.whl (491 kB)


In [5]:
!pip install sentencepiece

In [6]:
!pip install protobuf

In [8]:
!pip install transformers==4.33.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 27.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 121.3 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.1
    Uninstalling tokenizers-0.21.1:
      Successfully uninstalled tokenizers-0.21.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.51.3
    Uninstalling transformers-4.51.3:
      Successfully uninstalled transformers-4.51.3


In [9]:
!pip show transformers

Name: transformers
Version: 4.33.1
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /venv/main/lib/python3.10/site-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: peft


In [10]:
!git lfs install

Updated git hooks.
Git LFS initialized.


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import copy
import numpy as np

In [ ]:
torch.nn.Linear.reset_parameters = lambda x: None

model = AutoModelForCausalLM.from_pretrained('facebook/layerskip-llama2-7B', torch_dtype=torch.bfloat16, token="")
tokenizer = AutoTokenizer.from_pretrained('facebook/layerskip-llama2-7B', token="")

# model = AutoModelForCausalLM.from_pretrained('meta-llama/Llama-2-7b-hf', torch_dtype=torch.bfloat16, token="")
# tokenizer = AutoTokenizer.from_pretrained('meta-llama/Llama-2-7b-hf', token="")

device='cuda:0'

model = model.to(device).eval()

In [16]:
from huggingface_hub import login

login(token="")

In [19]:
SEED=42

INTERVAL = 1
MERGE_LAYERS = 5
HIGHEST_LAY = 31
LOWEST_LAY = 0
THRESHOLD = 0.45

In [ ]:
np.random.seed(SEED)
torch.manual_seed(SEED)

#### LaCo algorithm from original author implementation: https://github.com/yangyifei729/LaCo

In [20]:
from copy import deepcopy

def merge_layers_return_model(model, merge_base_lay, merge_layer_num):
    merge_layer_num = min(merge_layer_num, len(model.model.layers) - merge_base_lay - 1)
    
    model_copy = deepcopy(model)
    for diff_lay in range(merge_base_lay+1, merge_base_lay+1+merge_layer_num):      
        # gate_proj
        model_copy.model.layers[merge_base_lay].mlp.gate_proj.weight.data.add_(
            model.model.layers[diff_lay].mlp.gate_proj.weight.data - model_copy.model.layers[merge_base_lay].mlp.gate_proj.weight.data
        )
        # down_proj
        model_copy.model.layers[merge_base_lay].mlp.down_proj.weight.data.add_(
            model.model.layers[diff_lay].mlp.down_proj.weight.data - model_copy.model.layers[merge_base_lay].mlp.down_proj.weight.data
        )
        # up_proj
        model_copy.model.layers[merge_base_lay].mlp.up_proj.weight.data.add_(
            model.model.layers[diff_lay].mlp.up_proj.weight.data - model_copy.model.layers[merge_base_lay].mlp.up_proj.weight.data
        )
        
        # q_proj
        model_copy.model.layers[merge_base_lay].self_attn.q_proj.weight.data.add_(
            model.model.layers[diff_lay].self_attn.q_proj.weight.data - model_copy.model.layers[merge_base_lay].self_attn.q_proj.weight.data
        )

        # k_proj
        model_copy.model.layers[merge_base_lay].self_attn.k_proj.weight.data.add_(
            model.model.layers[diff_lay].self_attn.k_proj.weight.data - model_copy.model.layers[merge_base_lay].self_attn.k_proj.weight.data
        ) 
    
        # v_proj
        model_copy.model.layers[merge_base_lay].self_attn.v_proj.weight.data.add_(
            model.model.layers[diff_lay].self_attn.v_proj.weight.data - model_copy.model.layers[merge_base_lay].self_attn.v_proj.weight.data
        )
    
        # o_proj
        model_copy.model.layers[merge_base_lay].self_attn.o_proj.weight.data.add_(
            model.model.layers[diff_lay].self_attn.o_proj.weight.data - model_copy.model.layers[merge_base_lay].self_attn.o_proj.weight.data
        )        
                       
    for diff_lay in range(merge_base_lay + merge_layer_num, merge_base_lay, -1):
        del(model_copy.model.layers[diff_lay])

    return model_copy

In [21]:
llama_7B_copy_to_compress = copy.deepcopy(model)

In [22]:
def cal_last_hidden_sim(model1, model2, tokenizer, sents):
    sim_ls = []
    for s in sents:
        encoded_inputs = tokenizer(s, return_tensors='pt').to(device)
        with torch.no_grad():
            outputs1 = model1(**encoded_inputs, output_hidden_states=True)
        hidden_states1 = outputs1.hidden_states[-1] # (1, seq_len, hidden)
        with torch.no_grad():
            outputs2 = model2(**encoded_inputs, output_hidden_states=True)
        hidden_states2 = outputs2.hidden_states[-1] # (1, seq_len, hidden)
        sim_ls.append(torch.cosine_similarity(hidden_states1.squeeze(0).flatten().unsqueeze(0), hidden_states2.squeeze(0).flatten().unsqueeze(0)))
    sim_ls = [i.item() for i in sim_ls]
    print(sim_ls, np.mean(sim_ls))
    return np.mean(sim_ls)

In [23]:
lay = HIGHEST_LAY - MERGE_LAYERS
last_merge_flag = False

sents = [
    'Olearia ericoides is a species of flowering plant in the family Asteraceae ',
    'Dr. Walter Broadbent (1868–1951) was an English physician remembered ',
    'The Gowanus Memorial Artyard was a nonprofit, artist-organized ',
    'James Brelsford (19 December 1855 – 24 December 1924) was an English cricketer who ',
    'Mount Brennan () is a dome-shaped mountain, high, which is ',
    'Noel Kinsey (24 December 1925 – 20 May 2017) was a Welsh international ',
    'Bolex International S. A. is a Swiss manufacturer of motion picture cameras ',
    'Oblique Strategies (subtitled Over One Hundred Worthwhile Dilemmas) is a card-based method for promoting '
    'The Little Harbor 44 is an American sailboat that was designed by Ted Hood ',
    'Crosley-class high speed transports were high speed transport ships that served in the '
]

while lay >= LOWEST_LAY:
    print(lay)
    print('current model layer', len(llama_7B_copy_to_compress.model.layers))
    tmp_merged_model = merge_layers_return_model(llama_7B_copy_to_compress, lay, MERGE_LAYERS-1)
    sim_value = cal_last_hidden_sim(model, tmp_merged_model, tokenizer, sents)
    if sim_value > THRESHOLD:
        llama_7B_copy_to_compress = tmp_merged_model
        lay -= INTERVAL
        if lay >= len(llama_7B_copy_to_compress.model.layers):
            lay = len(llama_7B_copy_to_compress.model.layers) - 1 - MERGE_LAYERS
    else:
        lay -= 1
    

26
current model layer 32
[0.8125, 0.81640625, 0.859375, 0.86328125, 0.8515625, 0.8671875, 0.8203125, 0.8046875, 0.84765625] 0.8381076388888888
25
current model layer 28
[0.69140625, 0.54296875, 0.67578125, 0.494140625, 0.71484375, 0.50390625, 0.72265625, 0.5546875, 0.73828125] 0.6265190972222222
24
current model layer 26
[0.65234375, 0.490234375, 0.6328125, 0.4375, 0.69140625, 0.455078125, 0.68359375, 0.50390625, 0.703125] 0.5833333333333334
23
current model layer 25
[0.6015625, 0.458984375, 0.59375, 0.3984375, 0.66015625, 0.4140625, 0.6484375, 0.466796875, 0.6640625] 0.5451388888888888
22
current model layer 24
[0.5625, 0.4375, 0.5546875, 0.380859375, 0.6171875, 0.392578125, 0.61328125, 0.4375, 0.625] 0.5134548611111112
21
current model layer 23
[0.515625, 0.400390625, 0.51171875, 0.333984375, 0.58203125, 0.361328125, 0.56640625, 0.396484375, 0.58984375] 0.4730902777777778
20
current model layer 22
[0.49609375, 0.37890625, 0.490234375, 0.326171875, 0.546875, 0.33984375, 0.53515625, 0

In [24]:
llama_7B_copy_to_compress.model.layers

ModuleList(
  (0-17): 18 x LlamaDecoderLayer(
    (self_attn): LlamaAttention(
      (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
      (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
      (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
      (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
      (rotary_emb): LlamaRotaryEmbedding()
    )
    (mlp): LlamaMLP(
      (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
      (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
      (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
      (act_fn): SiLUActivation()
    )
    (input_layernorm): LlamaRMSNorm()
    (post_attention_layernorm): LlamaRMSNorm()
  )
)

In [25]:
llama_7B_copy_to_compress.config.num_hidden_layers = len(llama_7B_copy_to_compress.model.layers)
llama_7B_copy_to_compress

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-17): 18 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_

In [ ]:
encoded_inputs = tokenizer(['Hi, What is your name? My name is'], return_tensors='pt').to(device)

output = llama_7B_copy_to_compress.generate(
    **encoded_inputs,
    max_new_tokens=32,
    do_sample=False,     
    temperature=0     
)

output_text = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
print(output_text)

In [27]:
llama_7B_copy_to_compress.save_pretrained("laco-layerskip-llama2-7b")
tokenizer.save_pretrained("laco-layerskip-llama2-7b")

('laco-layerskip-llama2-7b/tokenizer_config.json',
 'laco-layerskip-llama2-7b/special_tokens_map.json',
 'laco-layerskip-llama2-7b/tokenizer.model',
 'laco-layerskip-llama2-7b/added_tokens.json',
 'laco-layerskip-llama2-7b/tokenizer.json')

In [ ]:
# llama_7B_copy_to_compress = AutoModelForCausalLM.from_pretrained('laco-llama2-7b', torch_dtype=torch.bfloat16, token="")
# tokenizer = AutoTokenizer.from_pretrained('meta-llama/Llama-2-7b-hf', token="")

In [ ]:
llama_7B_copy_to_compress.push_to_hub("sidsr/laco-layerskip-llama2-7b")
tokenizer.push_to_hub("sidsr/laco-layerskip-llama2-7b")